 # MPA-MLF, Lab 7 - Convolutional Neural Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


0. Import libraries

In [ ]:
from tensorflow import keras
from keras.models import Sequential
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
###################################
# Write your own code here #


import zipfile
import os
import pandas as pd
from PIL import Image
from keras.utils import image_dataset_from_directory


from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU
from keras.optimizers import Adamax,Adam, SGD
from keras.callbacks import EarlyStopping


###################################
font = {'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [ ]:
X_train=[]
for image in os.listdir("/content/drive/MyDrive/data_projet/train/train_data_unlabeled"):
  path="/content/drive/MyDrive/data_projet/train/train_data_unlabeled/"+str(image)
  im=Image.open(path)
  im1=np.asarray(im)
  im.close()
  im1=im1[:,:,:3]
  X_train.append(im1)

X_train=np.asarray(X_train)


X_test=[]
for image in os.listdir("/content/drive/MyDrive/data_projet/test/test_data_unlabeled"):
  path="/content/drive/MyDrive/data_projet/test/test_data_unlabeled/"+str(image)
  im=Image.open(path)
  im1=np.asarray(im)
  im.close()
  im1=im1[:,:,:3]
  X_test.append(im1)

X_test=np.asarray(X_test)




In [ ]:
X_train_scaled = X_train.astype('float32') / 255.0
X_test_scaled = X_test.astype('float32') / 255.0

y_train=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/data_projet/y_train.csv"))
y_train=y_train.drop("id", axis=1)

y_train_encoded = to_categorical(y_train, num_classes=4)


In [ ]:
#X_train_scaled[0][0][0]
X_train_scaled
#y_train_encoded[0]

In [ ]:
model = Sequential()
###################################
# Write your own code here #

model.add(Conv2D(64,kernel_size=(3, 3), activation='linear', input_shape=(45,51,3)))
model.add(MaxPooling2D(pool_size=(4, 4)))

model.add(Conv2D(128,kernel_size=(3, 3), activation='linear',input_shape=(45,51,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256,kernel_size=(3, 3), activation='linear',input_shape=(45,51,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='softmax'))
model.add(Dense(256, activation='linear'))
model.add(Dense(128, activation='linear'))
model.add(Dense(4, activation='softmax'))

###################################



###################################
# Write your own code here #

#model.add(Conv2D(64,kernel_size=(3, 3), activation='linear', input_shape=(45,51,3)))
#model.add(MaxPooling2D(pool_size=(4, 4)))
#model.add(Conv2D(128,kernel_size=(3, 3), activation='softmax',input_shape=(45,51,3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(256,kernel_size=(3, 3), activation='linear',input_shape=(45,51,3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
#model.add(Dense(4, activation='softmax'))

###################################


#### 2.2 Compile the model

In [ ]:

###################################
# Write your own code here #

optimizer = Adamax(learning_rate = 0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

###################################
#For some reason I can not launch it at that point without a crash
#model.summary()

### 3. Training stage

#### 3.1 Model training

In [ ]:
###################################
# Write your own code here #
callback = EarlyStopping(monitor='accuracy', patience=8)
history = model.fit(X_train_scaled, y_train_encoded, epochs=80, batch_size=170, validation_split = 0.2)#, callbacks=[callback])


###################################

Epoch 1/80
77/77 [==============================] - 1s 14ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3327 - val_accuracy: 0.3457
Epoch 2/80
77/77 [==============================] - 1s 10ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3326 - val_accuracy: 0.3457
Epoch 3/80
77/77 [==============================] - 1s 11ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3327 - val_accuracy: 0.3457
Epoch 4/80
77/77 [==============================] - 1s 11ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3326 - val_accuracy: 0.3457
Epoch 5/80
77/77 [==============================] - 1s 11ms/step - loss: 1.3359 - accuracy: 0.3320 - val_loss: 1.3324 - val_accuracy: 0.3457
Epoch 6/80
77/77 [==============================] - 1s 11ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3327 - val_accuracy: 0.3457
Epoch 7/80
77/77 [==============================] - 1s 11ms/step - loss: 1.3358 - accuracy: 0.3320 - val_loss: 1.3327 - val_accuracy: 0.3457
Epoch 8/80
77

#### 3.1 Model Prediction

In [ ]:
predictions = model.predict(X_test_scaled)
predictions

150/150 [==============================] - 0s 2ms/step


array([[0.12528478, 0.33525446, 0.26679593, 0.27266482],
       [0.13209116, 0.33207217, 0.2719996 , 0.2638371 ],
       [0.12082714, 0.3366919 , 0.26280427, 0.27967674],
       ...,
       [0.1069921 , 0.34051967, 0.24873444, 0.3037538 ],
       [0.1325792 , 0.3321072 , 0.27208534, 0.2632282 ],
       [0.13434239, 0.33125   , 0.2733511 , 0.2610565 ]], dtype=float32)

In [ ]:
###################################
 #CSV Creation
id = range(len(predictions))
answers=[]
for loop in predictions:
  max=0
  count=0
  for j in range(len(loop)):
    if loop[j]>max:
      max=loop[j]
      count=j
  answers.append(count)

df = pd.DataFrame({'id': id, 'target': answers})

df.to_csv('X_test.csv', index=False)

###################################